In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

# Define the cascade classifier
cascade_path = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + cascade_path)

if face_cascade.empty():
    raise IOError(f"Cannot load cascade classifier xml file at {cv2.data.haarcascades + cascade_path}")

# Define the face detection function
def detect_face(img):
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return face_img, face_rects

cap = cv2.VideoCapture(0)
ret, frame = cap.read()

if not ret:
    raise IOError("Cannot read from webcam! Please restart.")

frame, face_rects = detect_face(frame)
if len(face_rects) == 0:
    raise IOError("No face detected")

(face_x, face_y, w, h) = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)

roi = frame[face_y:face_y + h, face_x:face_x + w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

# Filter parameters
arr_sharpen = [0, -1, 0, -1, 5, -1, 0, -1, 0]
sharpen = np.array(arr_sharpen).reshape(3, 3).astype(np.float32)

arr_blur = [0.0625, 0.125, 0.0625, 0.125, 0.25, 0.125, 0.0625, 0.125, 0.0625]
blur = np.array(arr_blur).reshape(3, 3).astype(np.float32)

arr_outline = [-1, -1, -1, -1, 8, -1, -1, -1, -1]
outline = np.array(arr_outline).reshape(3, 3).astype(np.float32)

arr_emboss = [-2, -1, 0, -1, 1, 1, 0, 1, 2]
emboss = np.array(arr_emboss).reshape(3, 3).astype(np.float32)

arr_identity = [0, 0, 0, 0, 1, 0, 0, 0, 0]
identity = np.array(arr_identity).reshape(3, 3).astype(np.float32)

def choose_kernel(choice):
    if choice == '0':
        return sharpen
    elif choice == '1':
        return blur
    elif choice == '2':
        return outline
    elif choice == '3':
        return emboss
    else:
        print("Invalid choice. Using identity kernel.")
        return identity

def bo_loc():
    print("Choose 0 for Sharpen")
    print("Choose 1 for Blur")
    print("Choose 2 for Outline")
    print("Choose 3 for Emboss")
    choice = input("Please choose: ")
    return choose_kernel(choice)

kernel = bo_loc()
kernel_name = str(kernel).split()[0][1:]
print(f"Selected kernel ({kernel_name}):")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame, face_rects = detect_face(frame)
    if len(face_rects) > 0:
        (x, y, w, h) = tuple(face_rects[0])
        track_window = (x, y, w, h)

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        x, y, w, h = track_window

        fix_channel_r = np.convolve(frame[:,:,0].flatten(), kernel.flatten(), mode='same').reshape(frame[:,:,0].shape)
        fix_channel_g = np.convolve(frame[:,:,1].flatten(), kernel.flatten(), mode='same').reshape(frame[:,:,1].shape)
        fix_channel_b = np.convolve(frame[:,:,2].flatten(), kernel.flatten(), mode='same').reshape(frame[:,:,2].shape)
        fix = np.stack((fix_channel_r, fix_channel_g, fix_channel_b), axis=2).astype(np.uint8)

        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        mask[y:y+h, x:x+w] = 255

        frame_outside_face = cv2.bitwise_and(fix, fix, mask=cv2.bitwise_not(mask))
        frame_inside_face = cv2.bitwise_and(frame, frame, mask=mask)
        final_frame = cv2.add(frame_outside_face, frame_inside_face)

        img2 = cv2.rectangle(final_frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
        cv2.imshow('img', img2)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


Choose 0 for Sharpen
Choose 1 for Blur
Choose 2 for Outline
Choose 3 for Emboss


Please choose:  0


Selected kernel ([):
